In [ ]:
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings
from typing import List
from flair.visual.training_curves import Plotter

In [44]:
# initialize embeddings. This takes time to load.
embedding_types: List[TokenEmbeddings] = [
    # GloVe embeddings for arabic
    WordEmbeddings('ar'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

2019-04-16 15:15:39,482 this function is deprecated, use smart_open.open instead


In [41]:
columns = {0: 'text', 1: 'ner'}
data_folder = '/home/jupyter/data_ner_ANER/'
corpus: TaggedCorpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns, 
  train_file='train.txt', 
  dev_file='dev.txt')
dict_tmp = corpus.make_tag_dictionary('ner')
dict_tmp.item2idx

2019-04-16 15:14:55,864 Reading data from /home/jupyter/data_ner_ANER
2019-04-16 15:14:55,865 Train: /home/jupyter/data_ner_ANER/train.txt
2019-04-16 15:14:55,867 Dev: /home/jupyter/data_ner_ANER/dev.txt
2019-04-16 15:14:55,867 Test: None
load 10 % of the train data as test dataset


{b'<unk>': 0,
 b'O': 1,
 b'B-LOC': 2,
 b'B-ORG': 3,
 b'I-ORG': 4,
 b'B-PER': 5,
 b'I-PER': 6,
 b'I-LOC': 7,
 b'B-MISC': 8,
 b'I-MISC': 9,
 b'': 10,
 b'\xe2\x80\x8f': 11,
 b'<START>': 12,
 b'<STOP>': 13}

In [45]:

# 1. get the corpus
columns = {0: 'text', 1: 'ner'}
data_folder = '/home/jupyter/data_ner_ANER/'
corpus: TaggedCorpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns, 
  train_file='train.txt', 
  dev_file='dev.txt')

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)


# initialize sequence tagger
from flair.models import SequenceTagger

# have a relatively small hidden_size
tagger: SequenceTagger = SequenceTagger(hidden_size=64,
                                        dropout = 0.2,
                                        rnn_layers = 2,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)# initialize trainer
# 6. Initialize trainer
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric

from flair.optim import AdamW
# optimizer = optimizer(momentum=0.9, dampening=0.9)
trainer: ModelTrainer = ModelTrainer(tagger, corpus, optimizer=AdamW)


2019-04-16 15:15:52,203 Reading data from /home/jupyter/data_ner_ANER
2019-04-16 15:15:52,204 Train: /home/jupyter/data_ner_ANER/train.txt
2019-04-16 15:15:52,205 Dev: /home/jupyter/data_ner_ANER/dev.txt
2019-04-16 15:15:52,207 Test: None
load 10 % of the train data as test dataset


In [46]:
import os
# 7. start training
result_folder = '/home/jupyter/result_ANER_64_2_l01_w01_d02_adamw/'
trainer.train(result_folder,
              EvaluationMetric.MICRO_F1_SCORE,
              learning_rate=0.1,
              mini_batch_size=32,
              weight_decay = 0.1,
              max_epochs=200,
              checkpoint=False)


2019-04-16 15:16:05,516 ----------------------------------------------------------------------------------------------------
2019-04-16 15:16:05,517 Evaluation method: MICRO_F1_SCORE
2019-04-16 15:16:05,519 ----------------------------------------------------------------------------------------------------
2019-04-16 15:16:06,205 epoch 1 - iter 0/121 - loss 74.46890259
2019-04-16 15:16:17,729 epoch 1 - iter 12/121 - loss 27.51570760
2019-04-16 15:16:24,560 epoch 1 - iter 24/121 - loss 22.52636974
2019-04-16 15:16:31,475 epoch 1 - iter 36/121 - loss 20.34656063
2019-04-16 15:16:37,446 epoch 1 - iter 48/121 - loss 18.31041048
2019-04-16 15:16:42,967 epoch 1 - iter 60/121 - loss 17.49510449
2019-04-16 15:16:51,552 epoch 1 - iter 72/121 - loss 16.83685119
2019-04-16 15:16:56,907 epoch 1 - iter 84/121 - loss 16.17926962
2019-04-16 15:17:02,785 epoch 1 - iter 96/121 - loss 15.66285282
2019-04-16 15:17:09,724 epoch 1 - iter 108/121 - loss 15.28954564
2019-04-16 15:17:14,509 epoch 1 - iter 120

KeyboardInterrupt: 

In [ ]:
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_training_curves(os.path.join(result_folder, 'loss.tsv'))
plotter.plot_weights(os.path.join(result_folder, 'weights.txt'))